# Insert Seed
read the seed account usernames from the `config.json` file, get their ids and account details and then insert them into the db

In [ ]:
# required imports to access api_db, misc, misc.CONFIG, ...
import sys
sys.path = ['.', '..', '../..'] + sys.path
from collection import *

<hr>
<h1 align="center">driver code</h1>

In [ ]:
seed_in_db = api_db.col_users.find(
    {"depth":0, "screen_name": {"$exists": True}},
    {"screen_name": True, "_id": False}, no_cursor_timeout=True)
already_collected = [u["screen_name"] for u in seed_in_db] # typically this will not be too large
print("Found %d accounts already collected" % len(already_collected))

In [ ]:
usernames = misc.CONFIG["seed"]["usernames"]
new_usernames = list(set(usernames) - set(already_collected))
print("Only %d accounts left to fetch and insert" % len(new_usernames))
seed = get_users_from_usernames(usernames)
# if an error occurs it is logged to stdout, if number don't match then it is just a duplicate
print("got %d/%d accounts (%d not found, suspended or duplicate)" % (len(seed), len(usernames), len(usernames) - len(seed)))

In [ ]:
# convert to db format
seed = list(map(user_to_db_format, seed))

In [ ]:
# set their depth=0
for s in seed: s["depth"] = 0

In [ ]:
upsert_users(seed)

In [ ]:
print("DONE")